# Downloading Sentinel-2 Multispectral Imager data for lakes

The code here is for retrieving satellite data (Sentinel-2 MSI) for lakes in Lithuania. The code in section 1 is to get data around the measurement site for one lake and the code in the section 2 is for downloading data for many lakes.

In [1]:
pip install geopandas

In [3]:
import geopandas as gpd
# import and read a file with measurement sites in lakes and ponds (point shapefile)
fp = "LT_taskai.shp"
data = gpd.read_file(fp)
ez1 = data[11:12]
ez1

,OID_,FID_1,Pavadinima,Kodas,Savivaldyb,Platuma__m,Ilguma__m_,VT_kodas,UBR,Long0,Lat0,geometry
11,11,104,Gaviekas,LTL534,Elektrėnų sav.,6067815.619,539828.3952,LT110031152,Nemuno,24.62235,54.747144,POINT (24.62235 54.74714)


In [7]:
# authorization of account in Google Earth Engine
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qi3y4xpMfdrrfW5xvPRtNM5zX_QjWgjyVCgQV4Y-Da4&tc=3mB1bHMkljeUDdrac74NHwGKcXDCSATrZnLWBAMV8ZE&cc=Tuiu1kEB2stPl4AExhhBJw8eD5hpqt6Z8OCnU8-DDyo

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjdeQkhuggnEFWPIH15pgi2LMoWEg_IaGjcj9vhlYPJL9y3ieeuktk

Successfully saved authorization token.


In [8]:
ez1.keys()

Index(['OID_', 'FID_1', 'Pavadinima', 'Kodas', 'Savivaldyb', 'Platuma__m',
       'Ilguma__m_', 'VT_kodas', 'UBR', 'Long0', 'Lat0', 'geometry'],
      dtype='object')

In [9]:
## Set parameters
START_DATE = '2017-03-01' # retrieve data from
END_DATE = '2022-05-18'   # retrieve data until
CLOUD_FILTER = 60         # percentage of cloud cover in an image
CLD_PRB_THRESH = 40       # cloud probability threshold
NIR_DRK_THRESH = 0.15     # NIR dark threshold
CLD_PRJ_DIST = 1          # distance to cloud
BUFFER = 50               # cloud buffer

### 1 Get data for one lake

In [10]:
# Define area of interest - a rectangle around the measurement site point. 
# This should include at least 9 pixels of Sentinel-2 MSI data
# Lon - longitude of a measurement site point(coordinate system WGS 1984)
# Lat - latitude of a measurement site point (coordinate system WGS 1984)
AOI = ee.Geometry.Rectangle(
    [
     float(ez1.Long0) - 0.0002, float(ez1.Lat0) - 0.00013,
     float(ez1.Long0) + 0.0002, float(ez1.Lat0) + 0.00013
    ])

In [11]:
# Import and filter Sentinel-2 surface reflectance data (S2_SR collection)
s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
    .select(['B1',
                                          'B2',
                                          'B3',
                                          'B4',
                                          'B5',
                                          'B6',
                                          'B7',
                                          'B8',
                                          'B8A',
                                          'B9',
                                          'B11',
                                          'B12',
                                          'AOT',
                                          'WVP',
                                          'SCL',
                                          'TCI_R',
                                          'TCI_G',
                                          'TCI_B',
                                          'QA10',
                                          'QA20',
                                          'QA60'])             
    .filterBounds(AOI) ## select data in AOI
    .filterDate(START_DATE, END_DATE) ## filter data according to the selected time frame
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))) ## remove data from very cloudy tiles (60-100 %)

# Import and filter s2cloudless
s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
    .filterBounds(AOI)
    .filterDate(START_DATE, END_DATE))

## Intersect the data from both collections
s2_sr_col = s2_sr_col.map(lambda image: image.clip(AOI))
s2_cloudless_col = s2_cloudless_col.map(lambda image: image.clip(AOI))

# Join the filtered s2cloudless collection to the surface reflectance (SR) collection by the 'system:index' property.
s2_sr_cld_col_eval =  ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
    'primary': s2_sr_col,
    'secondary': s2_cloudless_col,
    'condition': ee.Filter.equals(**{
        'leftField': 'system:index',
        'rightField': 'system:index'
    })
}))

In [ ]:
## additional functions for filtering clouds and cloud shadows

In [12]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [13]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [14]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)
    #return img.addBands(is_cld_shdw)
 

In [15]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [16]:
# add shadow and cloud bands
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

In [ ]:
### Visualise all the added bands

In [17]:
# Mosaic the image collection.
img = s2_sr_cld_col_eval_disp.mosaic()

# Subset layers and prepare them for display.
clouds = img.select('clouds').selfMask()
shadows = img.select('shadows').selfMask()
dark_pixels = img.select('dark_pixels').selfMask()
probability = img.select('probability')
cloudmask = img.select('cloudmask').selfMask()
cloud_transform = img.select('cloud_transform')

# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=14)

# Add layers to the folium map.
m.add_ee_layer(img,
               {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
               'S2 image', True, 1, 9)
m.add_ee_layer(probability,
               {'min': 0, 'max': 100},
               'probability (cloud)', False, 1, 9)
m.add_ee_layer(clouds,
               {'palette': 'e056fd'},
               'clouds', False, 1, 9)
m.add_ee_layer(cloud_transform,
               {'min': 0, 'max': 1, 'palette': ['white', 'black']},
               'cloud_transform', False, 1, 9)
m.add_ee_layer(dark_pixels,
               {'palette': 'orange'},
               'dark_pixels', False, 1, 9)
m.add_ee_layer(shadows, {'palette': 'yellow'},
               'shadows', False, 1, 9)
m.add_ee_layer(cloudmask, {'palette': 'orange'},
               'cloudmask', True, 0.5, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [18]:
# set minimum working scale (m)
scale = 10
lst_r_poi = s2_sr_cld_col_eval_disp.getRegion(AOI, scale).getInfo()

In [19]:
# define a function to retrieve all filtered satellite data for one lake
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [20]:
# lst_df holds filtered satellite data for one lake
lst_df = ee_array_to_df(lst_r_poi,[
  'B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B8',
  'B8A',
  'B9',
  'B11',
  'B12',
  'AOT',
  'WVP',
  'SCL',
  'TCI_R',
  'TCI_G',
  'TCI_B',
  'QA10',
  'QA20',
  'QA60',
  'probability',
  'clouds',
  'dark_pixels',
  'cloud_transform',
  'shadows',
  'cloudmask'])
lst_df


,time,datetime,B1,B2,B3,B4,B5,B6,B7,B8,...,TCI_B,QA10,QA20,QA60,probability,clouds,dark_pixels,cloud_transform,shadows,cloudmask
0,1491125430459,2017-04-02 09:30:30.459,196,170,174,146,137,121,110,111,...,17,0,0,0,4,0,0,0,0,0
3,1495705233457,2017-05-25 09:40:33.457,251,198,208,202,184,89,96,103,...,20,0,0,0,2,0,1,0,0,0
5,1500025350000,2017-07-14 09:42:30.000,14725,13107,12090,11585,12110,11541,11344,11544,...,255,0,0,1024,100,1,0,1,0,1
6,1500629540081,2017-07-21 09:32:20.081,605,573,586,515,592,704,764,816,...,58,0,0,0,36,0,1,0,0,0
8,1505813430456,2017-09-19 09:30:30.456,56,149,204,173,149,86,72,71,...,15,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,1649669719277,2022-04-11 09:35:19.277,2359,1457,1625,1750,2328,2691,3087,1768,...,50,0,0,0,95,1,0,1,0,1
3104,1650965727699,2022-04-26 09:35:27.699,1157,1064,1075,1052,1044,1018,1024,1025,...,9,0,0,0,1,0,0,0,0,0
3105,1651657514640,2022-05-04 09:45:14.640,1127,1183,1174,1149,1145,1114,1129,1119,...,18,0,0,0,1,0,0,0,0,0
3106,1652089523476,2022-05-09 09:45:23.476,1214,1207,1249,1201,1215,1207,1228,1219,...,20,0,0,0,1,0,0,0,0,0


### 2 Downloading Sentinel-2 data for many lakes

In [21]:
from tqdm import tqdm

In [ ]:
temps = []

for i in tqdm(range(0, len(data))):
    if i % 10 == 0:
        print(i)
    else:
        print('.')
    
    ez2 = data[i:(i+1)]

    
    AOI = ee.Geometry.Rectangle(
    [
     float(ez1.Long0) - 0.0002, float(ez1.Lat0) - 0.00013,
     float(ez1.Long0) + 0.0002, float(ez1.Lat0) + 0.00013
    ])
    
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .select(['B1',
                                          'B2',
                                          'B3',
                                          'B4',
                                          'B5',
                                          'B6',
                                          'B7',
                                          'B8',
                                          'B8A',
                                          'B9',
                                          'B11',
                                          'B12',
                                          'AOT',
                                          'WVP',
                                          'SCL',
                                          'TCI_R',
                                          'TCI_G',
                                          'TCI_B',
                                          'QA10',
                                          'QA20',
                                          'QA60'])                      
        .filterBounds(AOI)
        .filterDate(START_DATE, END_DATE)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(AOI)
        .filterDate(START_DATE, END_DATE))

    s2_sr_col = s2_sr_col.map(lambda image: image.clip(AOI))
    s2_cloudless_col = s2_cloudless_col.map(lambda image: image.clip(AOI))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    s2_sr_cld_col_eval =  ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))  
    
    s2_sr_cld_col_eval_disp0 = s2_sr_cld_col_eval.map(add_cld_shdw_mask)
    
  
    try:
        scale = 10
        lst_r_poi = s2_sr_cld_col_eval_disp0.getRegion(AOI, scale).getInfo()

        lst_df = ee_array_to_df(lst_r_poi,[
                                              'B1',
                                              'B2',
                                              'B3',
                                              'B4',
                                              'B5',
                                              'B6',
                                              'B7',
                                              'B8',
                                              'B8A',
                                              'B9',
                                              'B11',
                                              'B12',
                                              'AOT',
                                              'WVP',
                                              'SCL',
                                              'TCI_R',
                                              'TCI_G',
                                              'TCI_B',
                                              'QA10',
                                              'QA20',
                                              'QA60',
                                              'probability',
                                              'clouds',
                                              'dark_pixels',
                                              'cloud_transform',
                                              'shadows',
                                              'cloudmask'])

        lst_df['Lake_ID'] = i

        temps.append(lst_df)

    except:
        print("An exception occurred at {}".format(i))     

  0%|          | 0/357 [00:00<?, ?it/s]

0


  0%|          | 1/357 [00:07<46:27,  7.83s/it]

.


  1%|          | 2/357 [00:14<41:02,  6.94s/it]

...

.


  2%|▏         | 6/357 [00:22<15:51,  2.71s/it]

.


  2%|▏         | 7/357 [00:24<14:56,  2.56s/it]

.


In [ ]:
# see the retreieved data
tmp = pd.concat(temps)
tmp.head()

In [ ]:
# add lake measurement site code data 
tmp['Kodas'] = data['Kodas'][tmp['Lake_ID']].values

In [ ]:
# Save the dataframe to csv
tmp.to_csv("Data_S2_LT.csv")